In [98]:
import pandas as pd
import ast
import re
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TextClassificationPipeline, pipeline
import transformers
from tqdm import tqdm
# Load the data
df  = pd.read_csv('./data/processed/cleaned_jobs.csv', sep=';')
cluster = pd.read_csv('./csv_files/similarity.csv', sep=',')
pipe = pipeline("token-classification", model="lm-ner-linkedin-skills-recognition")

In [100]:
for id in cluster.loc[cluster['cluster_kmean'] == 3]['id']:
    print(id)
    
    print(df.loc[df['id'] == id]['description'])


3734020007
132    application deadline november denmark location...
Name: description, dtype: object
3732853334
141    application deadline october denmark location ...
Name: description, dtype: object
3732218970
322    application deadline october denmark location ...
Name: description, dtype: object
3734213818
342    application deadline november denmark location...
Name: description, dtype: object
3739392611
463    application deadline october denmark location ...
Name: description, dtype: object
3732174140
739    application deadline october denmark location ...
Name: description, dtype: object
3732174143
747    application deadline october denmark location ...
Name: description, dtype: object
3733689511
806    application deadline november denmark location...
Name: description, dtype: object
3737169148
893    application deadline october denmark location ...
Name: description, dtype: object
3732220891
895    application deadline october denmark location ...
Name: description, dtyp

In [129]:
print(sorted(cluster['cluster_graph'].unique()))
for label in cluster['cluster_graph'].unique():
    print(label)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0
3
14
4
9
1
5
8
7
10
12
6
2
11
13


In [132]:
cluster_skill = {}
for label in tqdm(cluster['cluster_graph'].unique()):
    skills = []
    for id in cluster.loc[cluster['cluster_graph'] == label]['id']:
        skills.extend(pipe(df.loc[df['id'] == id]['description'].values[0]))
        
    skill_extract = [{'entity': entry['entity'], 'word': entry['word']} for entry in skills]
    cluster_skill[label] = skill_extract


100%|██████████| 15/15 [02:22<00:00,  9.47s/it]


In [163]:
cluster_skill[9]

[{'entity': 'B-TECHNOLOGY', 'word': 'electronics'},
 {'entity': 'B-TECHNOLOGY', 'word': 'hardware'},
 {'entity': 'B-BUS', 'word': 'construction'},
 {'entity': 'B-TECHNOLOGY', 'word': 'electronics'},
 {'entity': 'B-BUS', 'word': 'project'},
 {'entity': 'B-BUS', 'word': 'financing'},
 {'entity': 'B-BUS', 'word': 'propulsion'},
 {'entity': 'B-BUS', 'word': 'av'},
 {'entity': 'I-BUS', 'word': '##ion'},
 {'entity': 'I-BUS', 'word': '##ics'},
 {'entity': 'B-TECHNICAL', 'word': 'software'},
 {'entity': 'I-TECHNICAL', 'word': 'testing'},
 {'entity': 'B-SOFT', 'word': 'commercial'},
 {'entity': 'B-TECHNICAL', 'word': 'data'},
 {'entity': 'I-TECHNICAL', 'word': 'acquisition'},
 {'entity': 'B-TECHNOLOGY', 'word': 'control'},
 {'entity': 'I-TECHNOLOGY', 'word': 'system'},
 {'entity': 'B-TECHNOLOGY', 'word': 'hardware'},
 {'entity': 'B-SOFT', 'word': 'instrumentation'},
 {'entity': 'B-BUS', 'word': 'construction'},
 {'entity': 'B-TECHNICAL', 'word': 'troubles'},
 {'entity': 'I-TECHNICAL', 'word': '

In [146]:

pd.DataFrame.from_dict(cluster_skill.items()).to_csv("./csv_files/skills.csv", index=False)

In [99]:
def words_to_sentence(word_list):
    return " ".join(word_list)
def remove_words_with_numbers(word_list_str):
    """
    Takes a string representation of a list of words as input,
    removes any special characters from the words, and then removes any words that contain numbers.

    Args:
      word_list_str: A string representation of a list of words.

    Returns:
      The function `remove_words_with_numbers` returns a list of words without any special characters or
    numbers.
    """
    word_list = ast.literal_eval(word_list_str)
    word_list_without_special = [
        re.sub(r"[^a-zA-Z0-9\s]", "", word) for word in word_list
    ]
    word_list_without_numbers = [
        word for word in word_list_without_special if not re.search(r"\d", word)
    ]
    return word_list_without_numbers
df["description"] = df["description"].apply(
            lambda x: remove_words_with_numbers(x)
        )
df["description"] = df["description"].apply(words_to_sentence)